## 03. 다중선형회귀(Multivariable Linear Regression)

- 단순선형회귀(Simple Linear Regression): $x$가 1개인 선형회귀
- 다중선형회귀(Multivariable Linear Regression): 다수의 $x$로부터 $y$를 예측하는 선형회귀

$ H(x) = w_1x_1 + w_2x_2 + w_3x_3 + b $

(예) Quiz 세 번의 성적으로부터 기말 성적을 예측하는 모델을 만들고자 할 때

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [5]:
# 훈련 데이터 선언
x1_train = torch.FloatTensor([[73],
                              [93],
                              [89],
                              [96],
                              [73]])
x2_train = torch.FloatTensor([[80],
                              [88],
                              [91],
                              [98],
                              [66]])
x3_train = torch.FloatTensor([[75],
                              [93],
                              [90],
                              [100],
                              [70]])
y_train = torch.FloatTensor([[152],
                             [185],
                             [180],
                             [196],
                             [142]])

In [6]:
# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [8]:
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs):
    
    # H(x) 계산 = forward()
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b # 여기서는 일일이 x와 대응하는 w를 곱하도록 적어주었음 --> 개선예정!
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train)**2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 미분값이 누적되지 않도록 초기화해주고
    cost.backward() # 비용함수가 최솟값이 되는 지점의 parameter를 찾아야 하므로 비용함수에 대해 기울기 계산
    optimizer.step() # parameter 업데이트
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.718 w2: 0.613 w3: 0.680 b: 0.009 Cost: 1.079378
Epoch  100/1000 w1: 0.722 w2: 0.608 w3: 0.680 b: 0.009 Cost: 1.038584
Epoch  200/1000 w1: 0.727 w2: 0.603 w3: 0.681 b: 0.010 Cost: 0.999894
Epoch  300/1000 w1: 0.731 w2: 0.599 w3: 0.681 b: 0.010 Cost: 0.963217
Epoch  400/1000 w1: 0.735 w2: 0.595 w3: 0.681 b: 0.010 Cost: 0.928421
Epoch  500/1000 w1: 0.739 w2: 0.591 w3: 0.681 b: 0.010 Cost: 0.895453
Epoch  600/1000 w1: 0.743 w2: 0.586 w3: 0.682 b: 0.010 Cost: 0.864161
Epoch  700/1000 w1: 0.746 w2: 0.583 w3: 0.682 b: 0.010 Cost: 0.834503
Epoch  800/1000 w1: 0.750 w2: 0.579 w3: 0.682 b: 0.010 Cost: 0.806375
Epoch  900/1000 w1: 0.754 w2: 0.575 w3: 0.682 b: 0.010 Cost: 0.779696


### 3. 벡터와 행렬 연산으로 바꾸기

- 기존에는 x값과 w값이 각각 3개여서 하나하나씩 정의해주고, 또 곱셈을 실행할 수 있었지만... input값과 가중치의 개수가 엄청나게 많다면?!
- 이를 해결하기 위해 행렬 곱셈 연산(또는 벡터의 내적)을 사용한다.
- 행렬의 곱셈 과정에서 이루어지는 벡터 연산(행x열)을 벡터의 내적(dot product)이라고 한다.

#### 행렬/벡터 연산으로 이해하기

$ H(X) = w_1x_1 + w_2x_2 + w_3x_3 $

위 식은 $(x_1 x_2 x_3) \cdot (w_1 w_2 w_3).T$ 처럼 벡터 간의 내적으로 표현할 수 있다.
두 벡터를 각각 $X$와 $W$로 표현한다면, 가설은 다음과 같다.

$ H(X) = XW $

즉, x와 w의 원소의 개수에 상관없이 두 개의 변수만으로 표현 가능!

$X$의 행렬 사이즈가 (5 x 3) (sample의 개수는 5개, feature는 3개)라고 한다면,  
$W$의 벡터 사이즈는 (3 x 1)이고,  
$XW$의 결과가 (5 x 1)이 되므로 편향 벡터 $B$의 사이즈 또한 (5 x 1)이 된다.

$ H(X) = XW + B$

### 4. 행렬 연산을 고려하여 파이토치로 구현하기